In [14]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [10]:
novelties_services = pd.read_sql_table('mensajeria_novedadesservicio', engineSource)
dim_type_novelty = pd.read_sql_table('dim_type_novelty', engineDestination)
fact_novelty = novelties_services.groupby(['servicio_id', 'tipo_novedad_id']).size().reset_index(name='total_novedad')

fact_novelty.rename(columns={
    'servicio_id': 'service_id',
    'tipo_novedad_id': 'type_novelty_id'
}, inplace=True)
#fact_novelty = fact_novelty.merge(dim_type_novelty, on='type_novelty_id')
#fact_novelty["novelty_id"] = range(1, len(fact_novelty) + 1)
fact_novelty["saved"] = pd.to_datetime('now')

fact_novelty

,service_id,type_novelty_id,total_novedad,saved
0,29,1,1,2024-10-22 20:39:14.921306
1,29,2,1,2024-10-22 20:39:14.921306
2,51,1,5,2024-10-22 20:39:14.921306
3,51,2,3,2024-10-22 20:39:14.921306
4,57,1,2,2024-10-22 20:39:14.921306
...,...,...,...,...
3557,28455,1,1,2024-10-22 20:39:14.921306
3558,28461,2,1,2024-10-22 20:39:14.921306
3559,28464,2,1,2024-10-22 20:39:14.921306
3560,28466,2,1,2024-10-22 20:39:14.921306


In [17]:
fact_novelty.to_sql("fact_novelty", engineDestination, if_exists="replace", index_label="novelty_id")

alter_table_query = """
ALTER TABLE fact_novelty
ADD CONSTRAINT fk_type_novelty FOREIGN KEY (type_novelty_id) REFERENCES dim_type_novelty(type_novelty_id);
"""

with engineDestination.connect() as connection:
    connection.execute(text(alter_table_query))